In [1]:
words = open('../makemore/names.txt', 'r').read().splitlines()

In [2]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
min(len(w) for w in words)

2

In [4]:
max(len(w) for w in words)

15

In [5]:
b = {}
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1
#         print(ch1, ch2)

In [6]:
# sorted(b.items(), key = lambda kv: -kv[1])

In [7]:
import torch

In [8]:
N = torch.zeros((27,27), dtype=torch.int32)

In [9]:
chars = sorted(list(set(''.join(words))))

In [10]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}

In [11]:
# stoi

In [12]:
# itos

In [13]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1,ix2] += 1

In [14]:
# N

In [15]:
import matplotlib.pyplot as plt

In [16]:
# plt.imshow(N)

In [17]:
# plt.figure(figsize=(16,16))
# plt.imshow(N, cmap="Blues")
# for i in range(27):
#     for j in range(27):
#         chstr = itos[i] + itos[j] # character string - bigrams in character representation
#         plt.text(j, i, chstr, ha='center', va='bottom', color='gray') # plot the bigram text
#         plt.text(j, i, N[i, j].item(), ha='center', va='top', color='gray') # plot the no. of times a bigram occurs

# # plt.axis('off');

## Sampling from this char-level bigram count / probability model we built

In [18]:
ix = 0 # coz, we start with .
N[ix] # give 1-d array of starting char after .

# we have to sample from this array

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [28]:
P = (N+1).float() # +1 is for smoothing out the prob distribution
P /= P.sum(1, keepdims=True)

In [29]:
g = torch.Generator().manual_seed(2147483647)


for i in range(5):
    ix = 0
    out = []
    while True:
#         p = N[ix].float()
#         p = p/p.sum()
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

cexze.
momasurailezitynn.
konimittain.
llayn.
ka.


## Loss function

In [30]:
log_likelihood = 0.0
n = 0
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1,ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
#         print(f'{ch1}{ch2}: {prob:.4f}  {logprob:.4f}')
print(f'{log_likelihood = }')
nll = -log_likelihood
print(f'{nll = }')
print(f'{nll/n}') # loss function for the training set assigned by the model
# the lower it is the better off we are  

log_likelihood = tensor(-559951.5625)
nll = tensor(559951.5625)
2.4543561935424805


#### When all prob will be 1, the log_likelihood with become 0 (best)
- But by semantics, we want loss to be lower, so we take negative